todo integrado

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re
import os

# Function to convert degrees to degrees and minutes
def degrees_to_dm(degrees):
    d = degrees.astype(int)
    m = (degrees - d) * 60
    return np.column_stack((d, m))

# Function to convert day of year to date
def doy_to_date(year, doy):
    return pd.to_datetime(year * 1000 + doy, format='%Y%j')

# Function to process GPS data from Canadian Geodetic Service files
def process_gps_data(filename):
    leap_sec = 17

    with open(filename, 'r') as file:
        lines = file.readlines()

    data = []
    for line in lines:
        parts = line.split('\t')
        if len(parts) >= 7:  # Ensure we have enough parts
            try:
                timestamp = parts[0].strip()
                latitude = float(re.findall(r'[-+]?\d*\.\d+|\d+', parts[4])[0])
                longitude = float(re.findall(r'[-+]?\d*\.\d+|\d+', parts[5])[0])
                elevation = float(re.findall(r'[-+]?\d*\.\d+|\d+', parts[6])[0])
                data.append([timestamp, latitude, longitude, elevation])
            except (IndexError, ValueError):
                continue

    # Convert the collected data into a DataFrame
    df = pd.DataFrame(data, columns=['Timestamp', 'Latitude', 'Longitude', 'Elevation'])

    # Convert timestamps to datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M:%S.%f')
    df['Year'] = df['Timestamp'].dt.year
    df['DOY'] = df['Timestamp'].dt.dayofyear
    df['Hour'] = df['Timestamp'].dt.hour
    df['Minute'] = df['Timestamp'].dt.minute
    df['Second'] = df['Timestamp'].dt.second

    lat_dm = degrees_to_dm(df['Latitude'])
    lon_dm = degrees_to_dm(df['Longitude'])

    adjusted_time = df['Timestamp'] - timedelta(seconds=leap_sec)

    gps_time_date = {
        'Year': adjusted_time.dt.year,
        'Month': adjusted_time.dt.month,
        'Day': adjusted_time.dt.day,
        'Hour': adjusted_time.dt.hour,
        'Minute': adjusted_time.dt.minute,
        'Second': adjusted_time.dt.second
    }

    gps_positions = {
        'LatDegrees': lat_dm[:, 0],
        'LatMinutes': lat_dm[:, 1],
        'LongDegrees': lon_dm[:, 0],
        'LongMinutes': lon_dm[:, 1],
        'Elevation': df['Elevation']
    }

    return gps_positions, gps_time_date

# Function to convert geodetic coordinates to Southern Hemisphere polar stereographic coordinates
def geog_to_pol_wgs84_71S(lat, lon):
    pi = 3.141592653589793
    dtorad = pi / 180
    phi_c = -71
    lon0 = 0
    a = 6378137
    f = 1 / 298.257223563
    e2 = 2 * f - f ** 2
    e = np.sqrt(e2)
    m_c = np.cos(phi_c * dtorad) / np.sqrt(1 - e2 * (np.sin(phi_c * dtorad)) ** 2)
    t_c = np.tan(pi / 4 + phi_c * dtorad / 2) / ((1 + e * np.sin(phi_c * dtorad)) / (1 - e * np.sin(phi_c * dtorad))) ** (e / 2)
    t = (np.tan(pi / 4 + lat * dtorad / 2)) / ((1 + e * np.sin(lat * dtorad)) / (1 - e * np.sin(lat * dtorad))) ** (e / 2)
    rho = a * m_c * t / t_c
    x = -rho * (np.sin((lon0 - lon) * dtorad))
    y = rho * (np.cos((lon0 - lon) * dtorad))
    return [x, y]

# Function to calculate distances between GPS coordinates
def calculate_distances(long, lat):
    pi180 = np.pi / 180
    earth_radius = 6378.137e3

    long1 = long[:-1] * pi180
    long2 = long[1:] * pi180
    lat1 = lat[:-1] * pi180
    lat2 = lat[1:] * pi180

    dlon = long2 - long1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    dist = earth_radius * c

    return dist

# Main function to execute the script
def main():
    filename = "EVENTS.CSV"
    if not os.path.isfile(filename):
        print("File EVENTS.CSV not found. Please upload the file and try again.")
        return

    gps_positions, gps_time_date = process_gps_data(filename)

    longitudes = gps_positions['LongDegrees'] + gps_positions['LongMinutes'] / 60
    latitudes = gps_positions['LatDegrees'] + gps_positions['LatMinutes'] / 60
    distances = calculate_distances(np.array(longitudes), np.array(latitudes))

    print("GPS Positions:", gps_positions)
    print("GPS Time and Date:", gps_time_date)
    print("Distances between points:", distances)

if __name__ == "__main__":
    main()


GPS Positions: {'LatDegrees': array([   0., 7627., 2283., 7627., 2281., 7627., 7627., 2284., 2283.,
       7627., 7627., 2282., 2282., 7627., 7627., 2283., 7627., 2280.,
       7627., 2281., 7627., 7627., 2278., 2282., 7627., 7627., 2277.,
       2280., 7627., 7627., 2281., 7627., 2283., 7627., 2284., 7627.,
       7627., 2283., 2284., 7627., 7627., 2281., 2285., 7627., 7627.,
       2280.]), 'LatMinutes': array([-24.   ,  26.832,   0.   ,  26.832,   0.   ,  26.832,  26.832,
         0.   ,   0.   ,  26.832,  26.832,   0.   ,   0.   ,  26.832,
        26.832,   0.   ,  26.832,   0.   ,  26.832,   0.   ,  26.832,
        26.832,   0.   ,   0.   ,  26.832,  26.832,   0.   ,   0.   ,
        26.832,  26.832,   0.   ,  26.832,   0.   ,  26.832,   0.   ,
        26.832,  26.832,   0.   ,   0.   ,  26.832,  26.832,   0.   ,
         0.   ,  26.832,  26.832,   0.   ]), 'LongDegrees': array([    0., 10726.,  2282., 10726.,  2281., 10726., 10726.,  2281.,
        2281., 10726., 10726.,  2283., 

/esto me tiro Chat GTP

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re
import os
from google.colab import files

# Function to convert degrees to degrees and minutes
def degrees_to_dm(degrees):
    d = degrees.astype(int)
    m = (degrees - d) * 60
    return np.column_stack((d, m))

# Function to convert day of year to date
def doy_to_date(year, doy):
    return pd.to_datetime(year * 1000 + doy, format='%Y%j')

# Function to process dGPS data from Canadian Geodetic Service files
def process_gps_data(filename):
    leap_sec = 17

    with open(filename, 'r') as file:
        lines = file.readlines()

    data = []
    for line in lines:
        parts = line.split('\t')
        if len(parts) >= 7:  # Ensure we have enough parts
            try:
                timestamp = parts[0].strip()
                latitude = float(re.findall(r'[-+]?\d*\.\d+|\d+', parts[4])[0])
                longitude = float(re.findall(r'[-+]?\d*\.\d+|\d+', parts[5])[0])
                elevation = float(re.findall(r'[-+]?\d*\.\d+|\d+', parts[6])[0])
                data.append([timestamp, latitude, longitude, elevation])
            except (IndexError, ValueError):
                continue

    # Debug: Print the raw data collected
    print("Raw data collected:", data)

    # Convert the collected data into a DataFrame
    df = pd.DataFrame(data, columns=['Timestamp', 'Latitude', 'Longitude', 'Elevation'])

    # Debug: Print the DataFrame to check its contents
    print("DataFrame created:", df)

    # Convert timestamps to datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M:%S.%f')
    df['Year'] = df['Timestamp'].dt.year
    df['DOY'] = df['Timestamp'].dt.dayofyear
    df['Hour'] = df['Timestamp'].dt.hour
    df['Minute'] = df['Timestamp'].dt.minute
    df['Second'] = df['Timestamp'].dt.second

    lat_dm = degrees_to_dm(df['Latitude'])
    lon_dm = degrees_to_dm(df['Longitude'])

    adjusted_time = df['Timestamp'] - timedelta(seconds=leap_sec)

    gps_time_date = {
        'Year': adjusted_time.dt.year,
        'Month': adjusted_time.dt.month,
        'Day': adjusted_time.dt.day,
        'Hour': adjusted_time.dt.hour,
        'Minute': adjusted_time.dt.minute,
        'Second': adjusted_time.dt.second
    }

    gps_positions = {
        'LatDegrees': lat_dm[:, 0],
        'LatMinutes': lat_dm[:, 1],
        'LongDegrees': lon_dm[:, 0],
        'LongMinutes': lon_dm[:, 1],
        'Elevation': df['Elevation']
    }

    # Debug: Print the processed GPS positions and time/date
    print("Processed GPS positions:", gps_positions)
    print("Processed GPS time/date:", gps_time_date)

    return gps_positions, gps_time_date

# Function to convert geodetic coordinates to Southern Hemisphere polar stereographic coordinates
def geog_to_pol_wgs84_71S(lat, lon):
    pi = 3.141592653589793
    dtorad = pi / 180
    phi_c = -71
    lon0 = 0
    a = 6378137
    f = 1 / 298.257223563
    e2 = 2 * f - f ** 2
    e = np.sqrt(e2)
    m_c = np.cos(phi_c * dtorad) / np.sqrt(1 - e2 * (np.sin(phi_c * dtorad)) ** 2)
    t_c = np.tan(pi / 4 + phi_c * dtorad / 2) / ((1 + e * np.sin(phi_c * dtorad)) / (1 - e * np.sin(phi_c * dtorad))) ** (e / 2)
    t = (np.tan(pi / 4 + lat * dtorad / 2)) / ((1 + e * np.sin(lat * dtorad)) / (1 - e * np.sin(lat * dtorad))) ** (e / 2)
    rho = a * m_c * t / t_c
    x = -rho * (np.sin((lon0 - lon) * dtorad))
    y = rho * (np.cos((lon0 - lon) * dtorad))
    return [x, y]

# Function to calculate distances between GPS coordinates
def calculate_distances(long, lat):
    pi180 = np.pi / 180
    earth_radius = 6378.137e3

    long1 = long[:-1] * pi180
    long2 = long[1:] * pi180
    lat1 = lat[:-1] * pi180
    lat2 = lat[1:] * pi180

    dlon = long2 - long1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    dist = earth_radius * c

    return dist

# Main function to execute the script
def main():
    uploaded = files.upload()

    if not uploaded:
        print("No files uploaded. Please upload the files and try again.")
        return

    for filename in uploaded.keys():
        gps_positions, gps_time_date = process_gps_data(filename)

        # Check if gps_positions and gps_time_date are not empty
        if not gps_positions or not gps_time_date:
            print(f"No data found in the file {filename}.")
            continue

        longitudes = gps_positions['LongDegrees'] + gps_positions['LongMinutes'] / 60
        latitudes = gps_positions['LatDegrees'] + gps_positions['LatMinutes'] / 60
        distances = calculate_distances(np.array(longitudes), np.array(latitudes))

        # Check if distances array is not empty
        if len(distances) == 0:
            print(f"No distances calculated for the file {filename}.")
            continue

        # Create output directory if it doesn't exist
        output_dir = '/content/Output'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Save gps_positions to a CSV file
        gps_positions_df = pd.DataFrame(gps_positions)
        gps_positions_df.to_csv(os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_gps_positions.csv'), index=False)

        # Save gps_time_date to a CSV file
        gps_time_date_df = pd.DataFrame(gps_time_date)
        gps_time_date_df.to_csv(os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_gps_time_date.csv'), index=False)

        # Save distances to a text file
        np.savetxt(os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_distances.txt'), distances, delimiter=',')

        print(f"GPS Positions saved to {os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_gps_positions.csv')}")
        print(f"GPS Time and Date saved to {os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_gps_time_date.csv')}")
        print(f"Distances between points saved to {os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_distances.txt')}")

if __name__ == "__main__":
    main()


Saving D1.csv to D1.csv
Raw data collected: []
DataFrame created: Empty DataFrame
Columns: [Timestamp, Latitude, Longitude, Elevation]
Index: []
Processed GPS positions: {'LatDegrees': array([], dtype=object), 'LatMinutes': array([], dtype=object), 'LongDegrees': array([], dtype=object), 'LongMinutes': array([], dtype=object), 'Elevation': Series([], Name: Elevation, dtype: object)}
Processed GPS time/date: {'Year': Series([], Name: Timestamp, dtype: int32), 'Month': Series([], Name: Timestamp, dtype: int32), 'Day': Series([], Name: Timestamp, dtype: int32), 'Hour': Series([], Name: Timestamp, dtype: int32), 'Minute': Series([], Name: Timestamp, dtype: int32), 'Second': Series([], Name: Timestamp, dtype: int32)}
No distances calculated for the file D1.csv.
